## Imports

In [1]:
import numpy as np

In [2]:
class TreeNode:
    """
    Represents a single node in a decision tree.

    Attributes:
    - feature_index (int or None): Index of the feature to split on at this node.
        None if the node is a leaf.
    - threshold (float or None): Threshold value used to split the feature.
        None if the node is a leaf.
    - left (TreeNode or None): Left child node (samples that satisfy the split condition).
    - right (TreeNode or None): Right child node (samples that do not satisfy the condition).
    - value (int or float or None): Class label for classification or predicted value for 
        regression. Only set for leaf nodes.

    Methods:
    - is_leaf(): Returns True if the node is a leaf (i.e., has a predicted value).
    """

    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index  
        self.threshold = threshold          
        self.left = left                    
        self.right = right                  
        self.value = value                  

    def is_leaf(self):
        return self.value is not None
    


class DecisionTreeClassifier:

    def __init__(self, max_depth=3):
        self.max_depth = max_depth
        self.root = None                 

    # Gini impurity measures how "pure" a node is
    def _gini_impurity(self, y):
        _, counts = np.unique(y, return_counts=True)  # Count each class
        probs = counts / counts.sum()                 # Calculate class probabilities
        return 1 - np.sum(probs ** 2)                 # Gini formula

    # Find the best feature and threshold to split the data
    def _best_split(self, X, y):
        best_gini = float('inf')      # Initialize best Gini as infinity
        best_feature = None           # Track best feature index
        best_threshold = None         # Track best threshold for splitting

        for feature_index in range(X.shape[1]):  # Iterate through each feature
            sorted_indices = np.argsort(X[:, feature_index])  # Sort feature values
            X_sorted, y_sorted = X[sorted_indices], y[sorted_indices]

            # Try all possible split points between unique values
            for i in range(1, len(X)):
                threshold = (X_sorted[i - 1, feature_index] + X_sorted[i, feature_index]) / 2

                # Create masks for splitting
                left_mask = X[:, feature_index] <= threshold
                right_mask = X[:, feature_index] > threshold

                y_left, y_right = y[left_mask], y[right_mask]

                # Skip invalid splits
                if len(y_left) == 0 or len(y_right) == 0:
                    continue

                # Compute weighted Gini impurity of the split
                gini_left = self._gini_impurity(y_left)
                gini_right = self._gini_impurity(y_right)
                weighted_gini = (len(y_left) * gini_left + len(y_right) * gini_right) / len(y)

                # Update if current split is better
                if weighted_gini < best_gini:
                    best_gini = weighted_gini
                    best_feature = feature_index
                    best_threshold = threshold

        return best_feature, best_threshold  # Return best split

    # Recursively build the decision tree
    def _build_tree(self, X, y, depth):
        # If node is pure or max depth reached, return a leaf node
        if len(set(y)) == 1 or depth >= self.max_depth:
            leaf_value = np.bincount(y).argmax()  # Most common class
            return TreeNode(value=leaf_value)

        # Find best split
        feature_index, threshold = self._best_split(X, y)

        # If no split found, return a leaf
        if feature_index is None:
            leaf_value = np.bincount(y).argmax()
            return TreeNode(value=leaf_value)

        # Split the data based on the best threshold
        left_mask = X[:, feature_index] <= threshold
        right_mask = X[:, feature_index] > threshold

        # Recursively build left and right subtrees
        left_child = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_child = self._build_tree(X[right_mask], y[right_mask], depth + 1)

        # Return the decision node
        return TreeNode(feature_index, threshold, left_child, right_child)

    # Fit the tree to training data
    def fit(self, X, y):
        self.root = self._build_tree(X, y, 0)  # Start at depth 0

    # Predict one sample by traversing the tree
    def _predict_one(self, x, node):
        if node.is_leaf():
            return node.value  # Return class label if leaf
        if x[node.feature_index] <= node.threshold:
            return self._predict_one(x, node.left)  # Go left
        else:
            return self._predict_one(x, node.right)  # Go right

    # Predict multiple samples
    def predict(self, X):
        return np.array([self._predict_one(x, self.root) for x in X])

    # Print the tree structure
    def print_tree(self, node=None, depth=0):
        if node is None:
            node = self.root
        indent = "  " * depth  # Indentation for visualization
        if node.is_leaf():
            print(f"{indent}Leaf: Predict {node.value}")
        else:
            print(f"{indent}Feature[{node.feature_index}] <= {node.threshold}")
            self.print_tree(node.left, depth + 1)
            self.print_tree(node.right, depth + 1)



In [ ]:
import numpy as np



# Main class for the Decision Tree classifier
class DecisionTreeClassifierScratch:
    def __init__(self, max_depth=3):
        self.max_depth = max_depth          # Maximum depth of the tree
        self.root = None                    # Root node of the tree

    # Gini impurity measures how "pure" a node is
    def _gini_impurity(self, y):
        _, counts = np.unique(y, return_counts=True)  # Count each class
        probs = counts / counts.sum()                 # Calculate class probabilities
        return 1 - np.sum(probs ** 2)                 # Gini formula

    # Find the best feature and threshold to split the data
    def _best_split(self, X, y):
        best_gini = float('inf')      # Initialize best Gini as infinity
        best_feature = None           # Track best feature index
        best_threshold = None         # Track best threshold for splitting

        for feature_index in range(X.shape[1]):  # Iterate through each feature
            sorted_indices = np.argsort(X[:, feature_index])  # Sort feature values
            X_sorted, y_sorted = X[sorted_indices], y[sorted_indices]

            # Try all possible split points between unique values
            for i in range(1, len(X)):
                threshold = (X_sorted[i - 1, feature_index] + X_sorted[i, feature_index]) / 2

                # Create masks for splitting
                left_mask = X[:, feature_index] <= threshold
                right_mask = X[:, feature_index] > threshold

                y_left, y_right = y[left_mask], y[right_mask]

                # Skip invalid splits
                if len(y_left) == 0 or len(y_right) == 0:
                    continue

                # Compute weighted Gini impurity of the split
                gini_left = self._gini_impurity(y_left)
                gini_right = self._gini_impurity(y_right)
                weighted_gini = (len(y_left) * gini_left + len(y_right) * gini_right) / len(y)

                # Update if current split is better
                if weighted_gini < best_gini:
                    best_gini = weighted_gini
                    best_feature = feature_index
                    best_threshold = threshold

        return best_feature, best_threshold  # Return best split

    # Recursively build the decision tree
    def _build_tree(self, X, y, depth):
        # If node is pure or max depth reached, return a leaf node
        if len(set(y)) == 1 or depth >= self.max_depth:
            leaf_value = np.bincount(y).argmax()  # Most common class
            return TreeNode(value=leaf_value)

        # Find best split
        feature_index, threshold = self._best_split(X, y)

        # If no split found, return a leaf
        if feature_index is None:
            leaf_value = np.bincount(y).argmax()
            return TreeNode(value=leaf_value)

        # Split the data based on the best threshold
        left_mask = X[:, feature_index] <= threshold
        right_mask = X[:, feature_index] > threshold

        # Recursively build left and right subtrees
        left_child = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_child = self._build_tree(X[right_mask], y[right_mask], depth + 1)

        # Return the decision node
        return TreeNode(feature_index, threshold, left_child, right_child)

    # Fit the tree to training data
    def fit(self, X, y):
        self.root = self._build_tree(X, y, 0)  # Start at depth 0

    # Predict one sample by traversing the tree
    def _predict_one(self, x, node):
        if node.is_leaf():
            return node.value  # Return class label if leaf
        if x[node.feature_index] <= node.threshold:
            return self._predict_one(x, node.left)  # Go left
        else:
            return self._predict_one(x, node.right)  # Go right

    # Predict multiple samples
    def predict(self, X):
        return np.array([self._predict_one(x, self.root) for x in X])

    # Print the tree structure
    def print_tree(self, node=None, depth=0):
        if node is None:
            node = self.root
        indent = "  " * depth  # Indentation for visualization
        if node.is_leaf():
            print(f"{indent}Leaf: Predict {node.value}")
        else:
            print(f"{indent}Feature[{node.feature_index}] <= {node.threshold}")
            self.print_tree(node.left, depth + 1)
            self.print_tree(node.right, depth + 1)

# Example usage
X = np.array([
    [22], [25], [47], [52], [46], [56], [23], [24]
])
y = np.array([0, 0, 1, 1, 1, 1, 0, 0])  # Binary classification labels

# Create and train the classifier
clf = DecisionTreeClassifierScratch(max_depth=3)
clf.fit(X, y)

# Print the structure of the trained tree
clf.print_tree()
